Ideas:

* Add FFT channels to CNN (Tried, does not make much difference)
* Finetune CNN (with SGD slow learning rate)
* 5-fold CNN
* Extract Features from CNN (before FC) and do XGB
* TTA (tried, made it better)
* More augmenting, additional 45, 135, 315 degrees
* More augmenting, random rotations and flips
* Predict test data and train with test
* Train on all of the training data (no train-val split)
* Try a different combination of combine predictions
* Fine-tune on pre-trained models (Get rid of some top layers because input size is small)

In this notebook, add Test data predictions to Training

* [Create Dataset](#Create-Dataset)
* [Train - Val Split](#Train---Val-Split)
* [Data Augmentation](#Data-Augmentation)
* [Training](#Training)
* [Predict Test](#Predict-Test)

In [1]:
import numpy as np
import pandas as pd
from skimage import transform
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, LeakyReLU
from keras import losses, optimizers, callbacks
from keras import regularizers
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from scipy.signal import fftconvolve
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold, KFold
from scipy import fftpack
import matplotlib.pyplot as plt
%matplotlib inline
RANDOM_SEED = 43
np.random.seed(RANDOM_SEED)

Using TensorFlow backend.
/home/can/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Create Dataset 

In [2]:
# Train
df_train = pd.read_json('./data/train.json')

# Test
df_test = pd.read_json('./data/test.json')
df_test_labels = pd.read_csv('./submissions/ourbest_explorestack_2.csv')
df_test = pd.merge(df_test, df_test_labels, on='id')
iceberg_cond = (df_test['is_iceberg'] >= 0.9) & (df_test['is_iceberg'] <= 1.0)
not_iceberg_cond = (df_test['is_iceberg'] >= 0.0) & (df_test['is_iceberg'] <= 0.1)
df_test = df_test[iceberg_cond | not_iceberg_cond]

# Merge Train and Test
df = pd.concat([df_train, df_test])
df
print len(df)

4927


In [3]:
def normalize(im):
    im = im - np.mean(im)
    im = im / np.std(im)
    return im

def get_convolve(im1, im2):
    im1 = im1 - np.mean(im1)
    im2 = im2 - np.mean(im2)
    im_conv = fftconvolve(im1, im2[::-1, ::-1], mode='same')
    return normalize(im_conv)

In [4]:
X, y = [], []
for im_band1, im_band2, label in zip(df['band_1'], df['band_2'], df['is_iceberg']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)    
    # Preprocess
    # - Zero mean
    im_band1 -= np.mean(im_band1)
    im_band2 -= np.mean(im_band2)
    # - Normalize
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)    
    im = np.concatenate([im_band1, im_band2], axis=2)
#     im = np.concatenate([normalize(im_band1), normalize(im_band2), get_convolve(im_band1, im_band2)], axis=2)
    X.append(im)
    y.append(label)    
X = np.array(X)
y = np.array(y)
print 'X.shape:', X.shape
print 'y.shape:', y.shape

X.shape: (4927, 75, 75, 2)
y.shape: (4927,)


In [5]:
def bypass(x):
    return x

def h_flip(x):
    return x[:, :, ::-1, :]

def v_flip(x):
    return x[:, ::-1, :, :]

def hv_flip(x):
    return h_flip(v_flip(x))

def rot90(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 90), axis=0) for im in x], axis=0)

def rot180(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 180), axis=0) for im in x], axis=0)

def rot270(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 270), axis=0) for im in x], axis=0)

def rot45(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 45, mode='reflect'), axis=0) for im in x], axis=0)

def rot135(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 135, mode='reflect'), axis=0) for im in x], axis=0)

def rot315(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 315, mode='reflect'), axis=0) for im in x], axis=0)

aug_funcs = [bypass, 
             h_flip, v_flip, hv_flip,
             rot90, rot180, rot270]

In [6]:
# 
def get_random_split(size, train_ratio):
    indices = range(size)
    cutoff = int(round(train_ratio * size))
    np.random.shuffle(indices)
    train_i = indices[:cutoff]
    test_i = indices[cutoff:]
    return train_i, test_i

In [7]:
def get_model(input_shape):
        # Architecture
        model = Sequential()
        # Block 1
        model.add(Conv2D(32, kernel_size=(3, 3),
                         activation='relu',
                         input_shape=input_shape))
        model.add(Dropout(0.25))
        # Block 2
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        # Block 3
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        # Block 4
        model.add(Conv2D(256, (3, 3)))
        model.add(LeakyReLU())          
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))
        # FC
        model.add(Flatten())
        model.add(Dense(32))
        model.add(LeakyReLU())
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))

        # Compile the model
        loss = losses.binary_crossentropy
        optimizer = optimizers.Adam()
        metrics = ['accuracy']
        model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

        #     
        return model

# For - Loop

In [8]:
# Parameters
N_MODELS = 30
train_ratio = 0.5
MODEL_NUMBERS = range(1, N_MODELS + 1)
# For - Loop
for MODEL_NUMBER in MODEL_NUMBERS:
    print '*** MODEL:', MODEL_NUMBER, ' ***'
    # *** Train - Val Split ***
    train_i, val_i = get_random_split(len(X), train_ratio)

    #
    X_train, y_train = X[train_i], y[train_i]
    X_val, y_val = X[val_i], y[val_i]
    print 'train_i[:5]', train_i[:5]
    print 'val_i[:5]', val_i[:5]
    print 'X_train.shape:', X_train.shape
    print 'y_train.shape:', y_train.shape
    print 'X_val.shape:', X_val.shape
    print 'y_val.shape:', y_val.shape
    print 'np.mean(y_train):', np.mean(y_train)
    print 'np.mean(y_val):', np.mean(y_val)
    
    # *** Data Augmentation ***
    # Train
    X_train = np.concatenate([func(X_train) for func in aug_funcs], axis=0)
    y_train = np.concatenate([y_train] * len(aug_funcs))

    # Validation
    X_val = np.concatenate([func(X_val) for func in aug_funcs], axis=0)
    y_val = np.concatenate([y_val] * len(aug_funcs))

    # 
    print 'X_train.shape:', X_train.shape
    print 'y_train.shape:', y_train.shape
    print 'X_val.shape:', X_val.shape
    print 'y_val.shape:', y_val.shape
    
    # *** Training ***
    model = get_model(input_shape=(75, 75, 2))
    # Callbacks
    def get_lr(epoch):
        lr = (np.random.rand() * 4e-2 + 1e-7)
        lr = np.clip(lr, a_min=None, a_max=0.025)
        print 'lr:', lr
        return lr
    MODEL_PATH = './models/model16/model' + str(MODEL_NUMBER) + '.h5'
    m_q = 'val_loss'
    model_path = MODEL_PATH
    check_pt = callbacks.ModelCheckpoint(filepath=model_path, monitor=m_q, save_best_only=True, verbose=1)
    early_stop = callbacks.EarlyStopping(patience=10, monitor=m_q, verbose=1)
    reduce_lr = callbacks.ReduceLROnPlateau(patience=2, factor=0.33, monitor=m_q, verbose=1)
    schedule_lr = callbacks.LearningRateScheduler(get_lr)
    callback_list = [check_pt, early_stop, reduce_lr]
    # fit
    model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200, batch_size=32)

*** MODEL: 1  ***
train_i[:5] [4630, 1750, 2066, 4474, 4720]
val_i[:5] [2465, 2849, 2680, 1086, 2103]
X_train.shape: (2464, 75, 75, 2)
y_train.shape: (2464,)
X_val.shape: (2463, 75, 75, 2)
y_val.shape: (2463,)
np.mean(y_train): 0.433649795455
np.mean(y_val): 0.426573623244
X_train.shape: (17248, 75, 75, 2)
y_train.shape: (17248,)
X_val.shape: (17241, 75, 75, 2)
y_val.shape: (17241,)
Train on 17248 samples, validate on 17241 samples
Epoch 1/200
17248/17248 [==============================] - 16s 912us/step - loss: 0.3635 - acc: 0.2625 - val_loss: 0.1971 - val_acc: 0.2790
Epoch 2/200
17248/17248 [==============================] - 15s 866us/step - loss: 0.2105 - acc: 0.2924 - val_loss: 0.2146 - val_acc: 0.2782
Epoch 3/200
17248/17248 [==============================] - 15s 872us/step - loss: 0.1857 - acc: 0.2975 - val_loss: 0.1606 - val_acc: 0.2877
Epoch 4/200
17248/17248 [==============================] - 15s 860us/step - loss: 0.1712 - acc: 0.2990 - val_loss: 0.1507 - val_acc: 0.2910
Epoc

Epoch 5/200
17248/17248 [==============================] - 15s 895us/step - loss: 0.1544 - acc: 0.2943 - val_loss: 0.1517 - val_acc: 0.2972
Epoch 6/200
17248/17248 [==============================] - 15s 893us/step - loss: 0.1473 - acc: 0.2987 - val_loss: 0.1483 - val_acc: 0.2977
Epoch 7/200
17248/17248 [==============================] - 16s 899us/step - loss: 0.1457 - acc: 0.2991 - val_loss: 0.1412 - val_acc: 0.3009
Epoch 8/200
17248/17248 [==============================] - 16s 902us/step - loss: 0.1368 - acc: 0.3029 - val_loss: 0.1463 - val_acc: 0.2984
Epoch 9/200
17248/17248 [==============================] - 16s 905us/step - loss: 0.1317 - acc: 0.3039 - val_loss: 0.1536 - val_acc: 0.2996
Epoch 10/200
17216/17248 [============================>.] - ETA: 0s - loss: 0.1288 - acc: 0.3048Epoch 00010: val_loss did not improve

Epoch 00010: reducing learning rate to 0.000330000015674.
17248/17248 [==============================] - 16s 907us/step - loss: 0.1287 - acc: 0.3047 - val_loss: 0.16

Epoch 8/200
17248/17248 [==============================] - 16s 905us/step - loss: 0.1365 - acc: 0.3028 - val_loss: 0.1395 - val_acc: 0.2993
Epoch 9/200
17248/17248 [==============================] - 15s 892us/step - loss: 0.1314 - acc: 0.3050 - val_loss: 0.1474 - val_acc: 0.2996
Epoch 10/200
17248/17248 [==============================] - 15s 893us/step - loss: 0.1265 - acc: 0.3065 - val_loss: 0.1548 - val_acc: 0.2978
Epoch 11/200
17152/17248 [============================>.] - ETA: 0s - loss: 0.1223 - acc: 0.3085Epoch 00011: val_loss did not improve

Epoch 00011: reducing learning rate to 0.000330000015674.
17248/17248 [==============================] - 16s 907us/step - loss: 0.1222 - acc: 0.3090 - val_loss: 0.1597 - val_acc: 0.2960
Epoch 12/200
17248/17248 [==============================] - 15s 898us/step - loss: 0.1065 - acc: 0.3138 - val_loss: 0.1399 - val_acc: 0.3003
Epoch 13/200
17216/17248 [============================>.] - ETA: 0s - loss: 0.1022 - acc: 0.3158Epoch 00013: val_loss

Epoch 17/200
17248/17248 [==============================] - 16s 902us/step - loss: 0.0898 - acc: 0.3116 - val_loss: 0.1450 - val_acc: 0.3102
Epoch 18/200
17248/17248 [==============================] - 16s 920us/step - loss: 0.0882 - acc: 0.3130 - val_loss: 0.1488 - val_acc: 0.3080
Epoch 19/200
17216/17248 [============================>.] - ETA: 0s - loss: 0.0897 - acc: 0.3130Epoch 00019: val_loss did not improve

Epoch 00019: reducing learning rate to 0.000108900003252.
17248/17248 [==============================] - 16s 906us/step - loss: 0.0897 - acc: 0.3128 - val_loss: 0.1475 - val_acc: 0.3096
Epoch 20/200
17248/17248 [==============================] - 16s 903us/step - loss: 0.0849 - acc: 0.3141 - val_loss: 0.1467 - val_acc: 0.3085
Epoch 21/200
17216/17248 [============================>.] - ETA: 0s - loss: 0.0840 - acc: 0.3147Epoch 00021: val_loss did not improve

Epoch 00021: reducing learning rate to 3.59369999205e-05.
17248/17248 [==============================] - 15s 896us/step -

Epoch 18/200
17216/17248 [============================>.] - ETA: 0s - loss: 0.0952 - acc: 0.3051Epoch 00018: val_loss did not improve

Epoch 00018: reducing learning rate to 3.59369999205e-05.
17248/17248 [==============================] - 16s 913us/step - loss: 0.0953 - acc: 0.3052 - val_loss: 0.1331 - val_acc: 0.3169
Epoch 19/200
17248/17248 [==============================] - 16s 928us/step - loss: 0.0938 - acc: 0.3059 - val_loss: 0.1311 - val_acc: 0.3175
Epoch 20/200
17216/17248 [============================>.] - ETA: 0s - loss: 0.0925 - acc: 0.3059Epoch 00020: val_loss improved from 0.12915 to 0.12911, saving model to ./models/model16/model5.h5

Epoch 00020: reducing learning rate to 1.18592095896e-05.
17248/17248 [==============================] - 16s 923us/step - loss: 0.0924 - acc: 0.3060 - val_loss: 0.1291 - val_acc: 0.3174
Epoch 21/200
17248/17248 [==============================] - 16s 901us/step - loss: 0.0931 - acc: 0.3048 - val_loss: 0.1333 - val_acc: 0.3165
Epoch 22/200
17

...

17248/17248 [==============================] - 16s 914us/step - loss: 0.1127 - acc: 0.3101 - val_loss: 0.1602 - val_acc: 0.2977
Epoch 11/200
17216/17248 [============================>.] - ETA: 0s - loss: 0.1075 - acc: 0.3122Epoch 00011: val_loss did not improve

Epoch 00011: reducing learning rate to 0.000108900003252.
17248/17248 [==============================] - 16s 931us/step - loss: 0.1074 - acc: 0.3124 - val_loss: 0.1514 - val_acc: 0.3011
Epoch 12/200
17248/17248 [==============================] - 16s 939us/step - loss: 0.1008 - acc: 0.3149 - val_loss: 0.1530 - val_acc: 0.3000
Epoch 13/200
17184/17248 [============================>.] - ETA: 0s - loss: 0.0985 - acc: 0.3156Epoch 00013: val_loss did not improve

Epoch 00013: reducing learning rate to 3.59369999205e-05.
17248/17248 [==============================] - 16s 939us/step - loss: 0.0984 - acc: 0.3157 - val_loss: 0.1458 - val_acc: 0.3020
Epoch 14/200
17248/17248 [==============================] - 17s 958us/step - loss: 0.0969

Epoch 19/200
17248/17248 [==============================] - 16s 933us/step - loss: 0.1024 - acc: 0.3184 - val_loss: 0.1482 - val_acc: 0.2938
Epoch 20/200
17184/17248 [============================>.] - ETA: 0s - loss: 0.1003 - acc: 0.3188Epoch 00020: val_loss did not improve

Epoch 00020: reducing learning rate to 1.29146797917e-06.
17248/17248 [==============================] - 16s 947us/step - loss: 0.1003 - acc: 0.3189 - val_loss: 0.1472 - val_acc: 0.2942
Epoch 21/200
17248/17248 [==============================] - 16s 925us/step - loss: 0.1023 - acc: 0.3192 - val_loss: 0.1474 - val_acc: 0.2941
Epoch 00021: early stopping
*** MODEL: 27  ***
train_i[:5] [408, 1649, 1811, 2669, 4904]
val_i[:5] [4053, 4343, 3463, 3264, 3632]
X_train.shape: (2464, 75, 75, 2)
y_train.shape: (2464,)
X_val.shape: (2463, 75, 75, 2)
y_val.shape: (2463,)
np.mean(y_train): 0.435267229383
np.mean(y_val): 0.424955532623
X_train.shape: (17248, 75, 75, 2)
y_train.shape: (17248,)
X_val.shape: (17241, 75, 75, 2)
y_val

X_train.shape: (17248, 75, 75, 2)
y_train.shape: (17248,)
X_val.shape: (17241, 75, 75, 2)
y_val.shape: (17241,)
Train on 17248 samples, validate on 17241 samples
Epoch 1/200
17248/17248 [==============================] - 18s 1ms/step - loss: 0.3434 - acc: 0.2498 - val_loss: 0.1729 - val_acc: 0.3012
Epoch 2/200
17248/17248 [==============================] - 16s 928us/step - loss: 0.1756 - acc: 0.2835 - val_loss: 0.1572 - val_acc: 0.3032
Epoch 3/200
17248/17248 [==============================] - 16s 931us/step - loss: 0.1560 - acc: 0.2873 - val_loss: 0.1664 - val_acc: 0.3038
Epoch 4/200
17248/17248 [==============================] - 16s 931us/step - loss: 0.1476 - acc: 0.2904 - val_loss: 0.1692 - val_acc: 0.2983
Epoch 5/200
17248/17248 [==============================] - 16s 942us/step - loss: 0.1457 - acc: 0.2916 - val_loss: 0.1530 - val_acc: 0.3068
Epoch 6/200
17248/17248 [==============================] - 16s 932us/step - loss: 0.1322 - acc: 0.2945 - val_loss: 0.1457 - val_acc: 0.3072


17248/17248 [==============================] - 15s 879us/step - loss: 0.1646 - acc: 0.2981 - val_loss: 0.1507 - val_acc: 0.2943
Epoch 5/200
17248/17248 [==============================] - 15s 873us/step - loss: 0.1580 - acc: 0.2992 - val_loss: 0.1492 - val_acc: 0.2932
Epoch 6/200
17248/17248 [==============================] - 15s 889us/step - loss: 0.1487 - acc: 0.3014 - val_loss: 0.1509 - val_acc: 0.2964
Epoch 7/200
17248/17248 [==============================] - 16s 906us/step - loss: 0.1500 - acc: 0.3017 - val_loss: 0.1685 - val_acc: 0.2948
Epoch 8/200
17152/17248 [============================>.] - ETA: 0s - loss: 0.1406 - acc: 0.3034Epoch 00008: val_loss did not improve

Epoch 00008: reducing learning rate to 0.000330000015674.
17248/17248 [==============================] - 17s 963us/step - loss: 0.1409 - acc: 0.3029 - val_loss: 0.1589 - val_acc: 0.2908
Epoch 9/200
17248/17248 [==============================] - 14s 834us/step - loss: 0.1237 - acc: 0.3077 - val_loss: 0.1373 - val_acc:

Epoch 5/200
17248/17248 [==============================] - 15s 887us/step - loss: 0.1501 - acc: 0.2955 - val_loss: 0.1492 - val_acc: 0.3014
Epoch 6/200
17248/17248 [==============================] - 15s 866us/step - loss: 0.1450 - acc: 0.2980 - val_loss: 0.1574 - val_acc: 0.2979
Epoch 7/200
17248/17248 [==============================] - 15s 899us/step - loss: 0.1415 - acc: 0.2990 - val_loss: 0.1402 - val_acc: 0.3029
Epoch 8/200
17248/17248 [==============================] - 15s 858us/step - loss: 0.1314 - acc: 0.3014 - val_loss: 0.1391 - val_acc: 0.3044
Epoch 9/200
17248/17248 [==============================] - 14s 824us/step - loss: 0.1305 - acc: 0.3025 - val_loss: 0.1606 - val_acc: 0.3018
Epoch 10/200
17248/17248 [==============================] - 14s 794us/step - loss: 0.1261 - acc: 0.3048 - val_loss: 0.1461 - val_acc: 0.3020
Epoch 11/200
17216/17248 [============================>.] - ETA: 0s - loss: 0.1192 - acc: 0.3063Epoch 00011: val_loss did not improve

Epoch 00011: reducing le

# Predict Test

In [9]:
# Load test data
df = pd.read_json('./data/test.json')
X_test, y_test = [], []
for im_band1, im_band2 in zip(df['band_1'], df['band_2']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)    
    # Preprocess - zero mean
    im_band1 -= np.mean(im_band1)
    im_band2 -= np.mean(im_band2)
    # Preprocess - normalize
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)    
    im = np.concatenate([im_band1, im_band2], axis=2)
    X_test.append(im)    
X_test = np.array(X_test)
print 'X_test.shape:', X_test.shape

X_test.shape: (8424, 75, 75, 2)


In [10]:
y_test_p = 0
# weights = [0.25, 0.4 / 3, 0.35, 0.4 / 3, 0.4 / 3]
# weights = [0.2, 0.18, 0.2, 0.2, 0.22]
weights = [0.1] * N_MODELS
for i, w in zip(range(N_MODELS), weights):
    print i
    # Load the model
    MODEL_PATH = './models/model16/model' + str(i + 1) + '.h5'
    model = load_model(MODEL_PATH)
    # predict - tta    
    for func in aug_funcs:
        y_test_p += model.predict(func(X_test), verbose=1).flatten() * w
# y_test_p = y_test_p / (len(aug_funcs) * 5.0)
y_test_p = y_test_p / (len(aug_funcs) * sum(weights))

0
8424/8424 [==============================] - 2s 256us/step
1
8424/8424 [==============================] - 2s 269us/step
2
8424/8424 [==============================] - 2s 233us/step
3
8424/8424 [==============================] - 2s 286us/step
4
8424/8424 [==============================] - 2s 246us/step
5
8424/8424 [==============================] - 2s 222us/step
6
8424/8424 [==============================] - 2s 266us/step
7
8424/8424 [==============================] - 2s 269us/step
8
8424/8424 [==============================] - 3s 298us/step
9
8424/8424 [==============================] - 2s 273us/step
10
8424/8424 [==============================] - 2s 290us/step
11
8424/8424 [==============================] - 3s 299us/step
12
8424/8424 [==============================] - 3s 302us/step
13
8424/8424 [==============================] - 2s 294us/step
14
8424/8424 [==============================] - 2s 277us/step
15
8424/8424 [==============================] - 2s 252us/step
16
8424/8424 [====

8424/8424 [==============================] - 3s 300us/step
20
8424/8424 [==============================] - 2s 280us/step
21
8424/8424 [==============================] - 2s 255us/step
22
8424/8424 [==============================] - 2s 241us/step
23
8424/8424 [==============================] - 3s 320us/step
24
8424/8424 [==============================] - 3s 299us/step
25
8424/8424 [==============================] - 3s 302us/step
26
8424/8424 [==============================] - 3s 310us/step
27
8424/8424 [==============================] - 3s 309us/step
28
8424/8424 [==============================] - 3s 319us/step
29
8424/8424 [==============================] - 2s 212us/step


In [11]:
df_sub = pd.DataFrame()
df_sub['id'] = df['id']
df_sub['is_iceberg'] = y_test_p.flatten()
df_sub.to_csv('./submissions/sub33.csv', index=False)